In [2]:
 ! pip install tensorflow tensorflow-text==2.8.* 
 ! pip install tf-models-official


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 46.4 MB/s 
     |████████████████████████████████| 238 kB 51.9 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 116 kB 49.6 MB/s 
     |████████████████████████████████| 4.6 MB 22.9 MB/s 
     |████████████████████████████████| 636 kB 54.6 MB/s 
     |████████████████████████████████| 1.1 MB 62.6 MB/s 
     |████████████████████████████████| 352 kB 55.5 MB/s 
     |████████████████████████████████| 99 kB 8.3 MB/s 
     |████████████████████████████████| 511.8 MB 7.7 kB/s 
     |████████████████████████████████| 1.6 MB 54.1 MB/s 
     |████████████████████████████████| 5.8 MB 34.7 M

Import packages


In [3]:
import pandas as pd
import numpy as np
import re
import os
import shutil
import tensorflow as tf
from tensorflow import keras
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow_text as tf_text
import tensorflow_hub as hub
from official.nlp import optimization
import csv



In [4]:
BATCH_SIZE=32
MAX_LENGTH=512
BUFFER_SIZE=50000

## Define source csv location

Tip: If the input files/folders change from the google drive, disconnect and reconnect the run time 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
input_file_path= "/content/drive/MyDrive/Colab Notebooks/training_140_sentiment.csv"
input_sentiment_train_dir="/content/drive/MyDrive/Colab Notebooks/sentiment/train"
input_pos_sentiment_dir="/content/drive/MyDrive/Colab Notebooks/sentiment/train/pos"
input_neg_sentiment_dir="/content/drive/MyDrive/Colab Notebooks/sentiment/train/neg"

In [7]:
os.path.isdir(input_pos_sentiment_dir)
os.path.isdir(input_neg_sentiment_dir)

True

### Use tf.data.datasets to convert tensors

In [ ]:
x=tf.range(10)
x

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

#### covert a tensor into a dataset(td.Dataset.dataset)

In [ ]:
# convert a tensor into a dataset
# creates every element(of the tensor) into a tensor
dataset=tf.data.Dataset.from_tensor_slices(x)
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [ ]:
for item in dataset:
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [ ]:
#repeating the dataset to create twice the number of element tensors 
dataset=dataset.repeat(2)
for item in dataset:
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [ ]:
# when using batch the element tensors are re-shaped with the dimension of the size specified. Induvidual tensors are merged to create a single tensor based on the batch size.
# here a batch size of 5 is used, on a dataset of 20 element tensors, which creates 4 tensors with 5 elements each
dataset_b5=dataset.batch(5)
for item in dataset_b5:
  print(item)

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int32)


In [ ]:
# when using a bath size of 7, the 20 element datset is reshaped into 2 tensors of 7 elements and a single tensor of the remaining 6 elements
dataset_b7=dataset.batch(7)
for item in dataset_b7:
  print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9], shape=(6,), dtype=int32)


In [ ]:
#use shuffle with buffer size
dataset_b8=dataset.batch(4).shuffle(buffer_size=3)
for item in dataset_b8:
  print(item)

tf.Tensor([4 5 6 7], shape=(4,), dtype=int32)
tf.Tensor([8 9 0 1], shape=(4,), dtype=int32)
tf.Tensor([0 1 2 3], shape=(4,), dtype=int32)
tf.Tensor([6 7 8 9], shape=(4,), dtype=int32)
tf.Tensor([2 3 4 5], shape=(4,), dtype=int32)


In [ ]:
#convert numpy to dataset
n=np.random.randint(low=1,high=10,size=10)
print(type(n))

<class 'numpy.ndarray'>


In [ ]:
dataset_n=tf.data.Dataset.from_tensor_slices(n)
for item in dataset_n:
  print(item)

tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)


## Use tf.datsets to read from input file

In [ ]:
# read the csv file using TextLineDataset

tweet_df=tf.data.TextLineDataset(input_file_path)

In [ ]:
tf.data.experimental.cardinality(tweet_df)

<tf.Tensor: shape=(), dtype=int64, numpy=-2>

In [ ]:
def ds_rec_count(df):
  count=0
  for rec in df.as_numpy_iterator():
    count+=1
  return count

In [ ]:
tweet_df_count=ds_rec_count(tweet_df)

In [ ]:
tweet_df_count

1600000

In [ ]:
for line in tweet_df:
  print(line)
  break

tf.Tensor(b'"0","1467810369","Mon Apr 06 22:19:45 PDT 2009","NO_QUERY","_TheSpecialOne_","@switchfoot http://twitpic.com/2y1zl - Awww, that\'s a bummer.  You shoulda got David Carr of Third Day to do it. ;D"', shape=(), dtype=string)


In [ ]:
#creating a key value for the label and features and using a dictionary to tag the features and label values
def extract_line_data_dict(line):
  line=tf.io.decode_csv(line,record_defaults=['string','string'],select_cols=[0,5])
  line_dict=dict(zip(['label','features'],line))
  return line_dict

In [ ]:
#returning the features and label fields
def extract_line_data(line):
  line=tf.io.decode_csv(line,record_defaults=['string','string'],select_cols=[0,5])
  return line

In [ ]:
tweet_dataset=tweet_df.map(extract_line_data)

In [ ]:
tweet_dataset


<MapDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [ ]:
#display elements in the dataset
for label,feature in tweet_dataset:
  print(label)
  print(feature)
  break

tf.Tensor(b'0', shape=(), dtype=string)
tf.Tensor(b"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", shape=(), dtype=string)


In [ ]:
def update_label(label,features):
  if label=='4':
    label=1
  else:
    label=0
  return ( features,label)

In [ ]:
tweet_dataset_label=tweet_dataset.map(update_label)

In [ ]:
for feature,label in tweet_dataset_label.batch(32):
  print(feature)
  print(label)
  break

tf.Tensor(
[b"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
 b"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
 b'@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'
 b'my whole body feels itchy and like its on fire '
 b"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "
 b'@Kwesidei not the whole crew ' b'Need a hug '
 b"@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?"
 b"@Tatiana_K nope they didn't have it " b'@twittera que me muera ? '
 b"spring break in plain city... it's snowing "
 b'I just re-pierced my ears '
 b"@caregiving I couldn't bear to watch it.  And I thought the UA loss was embarrassing . . . . ."
 b'@octolinz16 It it counts, idk why I did either. you never talk to me anymore '
 b"@smar

In [ ]:
#Shuffle the dataset
#tweet_dataset_label=tweet_dataset_label.batch(BATCH_SIZE)

In [ ]:
tweet_dataset_label.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None))

(TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [ ]:
for feature,label in tweet_dataset_label.take(1):
  print(feature)
  print(label)
  break

tf.Tensor(b"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
tweet_df_count

1600000

In [ ]:
train_len=int(0.7*tweet_df_count)

In [ ]:
train_len

1120000

In [ ]:
train_tweet_ds=tweet_dataset_label.take(train_len)


In [ ]:
train_tweet_ds


<TakeDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [ ]:
train_ds_count=ds_rec_count(train_tweet_ds)

In [ ]:
train_ds_count

1120000

In [ ]:
for feature,label in train_tweet_ds.take(1):
  print(feature)
  print(label)
  break

tf.Tensor(b"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
val_tweet_ds=tweet_dataset_label.skip(train_len)


In [ ]:
val_tweet_ds

<SkipDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [ ]:
#val_ds_count=ds_rec_count(val_tweet_ds)

In [ ]:
#val_ds_count

NameError: ignored

In [ ]:
val_tweet_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int32, name=None))

In [ ]:
for feature,label in remain_ds.take(1):
  print(feature)
  print(label)
  break

The input file is read using the TextLineDataset api, however there is no option to split the input into a train/validation/test set . (this is provided by the tf.keras.utils.text_dataset_from_directory api, however the input files are expected to be split by class when using this api ( meaning separate sub directories for each class, that stores the input files)
So the option to try would be to read the input file into a pandas dtaframe, use the train_test_split sklearn function to split into train and test and then convert into tensors, for use with the tensor tokensizer and the model

## Read input file using pandas

In [ ]:
sentiment_colls=['sentiment','tweet_id','tweet_date','query','username','tweet']
tweet_df=pd.read_csv(input_file_path,encoding='cp1252',names=sentiment_colls,usecols=['sentiment','tweet'])

In [ ]:
tweet_df.rename(columns={'sentiment':'label','tweet':'text'},inplace=True)

In [ ]:
tweet_df.shape



(1600000, 2)

In [ ]:
tweet_df.dtypes

label     int64
text     object
dtype: object

In [ ]:
tweet_df['label']=tweet_df['label'].map(lambda y: 1 if y==4 else y)

In [ ]:
X=tweet_df['text']
y=tweet_df['label']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [ ]:
type(X_train)

pandas.core.series.Series

In [ ]:
print("X Train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)


X Train (1280000,)
X_test (320000,)
y_train (1280000,)
y_test (320000,)


In [ ]:
X_train_ds=tf.convert_to_tensor(X_train)
y_train_ds=tf.convert_to_tensor(y_train )
X_test_ds=tf.convert_to_tensor(X_test )
y_test_ds=tf.convert_to_tensor(y_test )

## Use the tensorflow hub for Bert model/pre-process


In [8]:
tfhub_handle_encoder='https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [9]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model=hub.KerasLayer(tfhub_handle_encoder)

### Testing the BERT preprocessing layer

In [ ]:
##Testing to see the pre processing layer output
sample_text=['A small step for man, a giant leap for mankind']
preprocess_sample=bert_preprocess_model(sample_text)

There are  3 outputs from the preprocessing that a BERT model would use (input_words_id, input_mask and input_type_ids).

Some other important points:

The input is truncated to 128 tokens. 
The input_type_ids only have one value (0) because this is a single sentence input. For a multiple sentence input, it would have one number for each input.

The input mask id is used to identify which tokens to ignore(0) and which ones to use(1)- similar to attention mask

Since this text preprocessor is a TensorFlow model, It can be included in the model directly.

In [ ]:
print(preprocess_sample.keys())
print(preprocess_sample['input_type_ids'].shape)
print(preprocess_sample['input_mask'].shape)
print(preprocess_sample['input_word_ids'].shape)
print(preprocess_sample['input_type_ids'])
print(preprocess_sample['input_mask'])
print(preprocess_sample['input_word_ids'])

dict_keys(['input_mask', 'input_type_ids', 'input_word_ids'])
(1, 128)
(1, 128)
(1, 128)
tf.Tensor(
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 128), dtype=int32)
tf.Tensor(
[[1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 128), dtype=int32)
tf.Tensor(
[[  101  1037  2235  3357  2005  2158  1010  1037  5016 11679  2005 14938
    102     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0   

When using the transformers bert  tokenizer, the input needs to be a string or a list. 
To implement this, the input file is read into a pandas dataframe and the map function on the tokenzier is used. However when input data is large, this causes an out of memory error.
As an alternative, the file was read using tf.data.TextLineDataset , however the transformer tokenizer expects text or a list as input, not a tensor


In [ ]:
bert_model=hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
bert_results=bert_model(preprocess_sample)

The BERT models return a map with 3 important keys: pooled_output, sequence_output, encoder_outputs:

pooled_output  : represents each input sequence as a whole. The shape is [batch_size, H]. You can think of this as an embedding for the entire movie review.  
sequence_output :   represents each input token in the context. The shape is [batch_size, seq_length, H]. You can think of this as a contextual embedding for every token in the movie review.  

encoder_outputs :   are the intermediate activations of the L Transformer blocks. outputs["encoder_outputs"][i] is a Tensor of shape [batch_size, seq_length, 1024] with the outputs of the i-th Transformer block, for 0 <= i < L. The last value of the list is equal to sequence_output.
For the fine-tuning you are going to use the pooled_output array.

In [ ]:
print(bert_results.keys())
print(bert_results["pooled_output"].shape)
print(bert_results["sequence_output"].shape)
for ind,layer in enumerate(bert_results["encoder_outputs"]):
  print(ind,layer.shape)


### Using the preprocessing on the X_train

In [ ]:
preprocess_text=bert_preprocess_model(X_train)

In [ ]:
print(preprocess_text.keys())
print(preprocess_text['input_type_ids'].shape)
print(preprocess_text['input_mask'].shape)
print(preprocess_text['input_word_ids'].shape)
print(preprocess_text['input_type_ids'])
print(preprocess_text['input_mask'])
print(preprocess_text['input_word_ids'])

In [ ]:
preprocess_text=bert_preprocess_model(X_train_ds)

In [ ]:
print(preprocess_text.keys())
print(preprocess_text['input_type_ids'].shape)
print(preprocess_text['input_mask'].shape)
print(preprocess_text['input_word_ids'].shape)
print(preprocess_text['input_type_ids'])
print(preprocess_text['input_mask'])
print(preprocess_text['input_word_ids'])

## IMDB dataset

In [ ]:
#download imdb dataset
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [ ]:
#split into train,validation, test datasets
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds.element_spec

In [ ]:
for text,label in raw_train_ds:
  print('text',text)
  print('label',label)
  break

In [ ]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')

In [ ]:
train_ds.element_spec

In [ ]:
tweet_dataset_label.element_spec

## Read data from directory files


In [10]:
#split into train,validation, test datasets
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

The subdirectories for the multiple labels are expected to have one tweet or feedback per file. If there are not adequate files( like if we placed 2 files in positive labelled subdirectory- the text_dataset_from_directory throws an error. In the case where we used 10 files one for each tweet, 80% were used for training and 20% for validation

Rules:

1) Name the subdirectories by class ('pos' for positive and 'neg' for negetive for example)  
2) Move the text files ( one per input) into the class based subdirectories  
3) If the number of files are <=2 per class, when creating the validation split paramter- say using a 20-80% split on validation-training, the api throws an error that ' text file could not be found in the directory'

Convert each line of the inout file sentiment 140 into a separate file in a directory based on the label. Here input file is of the following format-
Line 1 of the .csv file:

"0","1467810369","Mon Apr 06 22:19:45 PDT 2009","NO_QUERY","_TheSpecialOne_","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

The first field is the label( value of "0" (negetive) or "4"(postive)
The tweet text is field 5. There are a total of 1.6 million tweets in the dataset, out of which the first 100,000 positive and negetive tweets would each be written to a separate file.


In [ ]:
# convert every line of input into separate files for upto 100,000 positive and negetive tweets
def split_line_to_file():
	neg_counter=0
	pos_counter = 0
	with open(input_file_path,"r",encoding='cp1252') as inpf:
		csv_reader=csv.reader(inpf,delimiter=',',quotechar='"')
	
		for line in csv_reader:
				if line[0]=='0' and neg_counter<=10000:
					neg_file=os.path.join(input_neg_sentiment_dir,'neg_text_'+str(neg_counter)+'.txt')
					with open(neg_file,'w') as negf:
						negf.write(line[5] + '\n')
					neg_counter+=1
				elif line[0]!='0' and pos_counter<=10000:
					pos_file = os.path.join(input_pos_sentiment_dir, 'pos_text_' + str(pos_counter) + '.txt')
					with open(pos_file,'w') as posf:
						posf.write(line[5] + '\n')
					pos_counter+=1
			
					

In [ ]:
# call function to split input file lines into separate files
#split_line_to_file()


In [10]:
input_pos_sentiment_dir

'/content/drive/MyDrive/Colab Notebooks/sentiment/train/pos'

In [11]:
#verify file counts
print("count of postive label files",len(os.listdir(input_pos_sentiment_dir)))


count of postive label files 10001


In [12]:
print("count of negetive label files",len(os.listdir(input_neg_sentiment_dir)))

count of negetive label files 10001


In [11]:
# train dataset creation
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    input_sentiment_train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
    )

Found 20002 files belonging to 2 classes.
Using 16002 files for training.


In [14]:
raw_train_ds.class_names

['neg', 'pos']

In [15]:
for i,label in enumerate(raw_train_ds.class_names):
  print(i," is class",label)

0  is class neg
1  is class pos


In [16]:
raw_train_ds.element_spec

(TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

In [17]:
for text,label in raw_train_ds.take(1):
  print('text:',text)
  print('label:' ,label)

text: tf.Tensor(
[b'so sleeepyy.. finally the load of laundry i had to get done is DONE  good night everyone\n'
 b'had fun fun fun last night   but now has to do coursework booooooo\n'
 b'Nobody wants to play with me. \n'
 b"@judyk113 Yeah, we're going to have neither, but it's all good. Hope you do. \n"
 b'going out sidee in dee cold. \n'
 b'@billwixey Wixey!!! Mark makes me coffee, brings me breakfast, pulls out my chair, keeps the temp in the studio a balmy 68. What about u? \n'
 b'@kirisutegomen4 lol yeah aus. Trivium fan right here \n'
 b'Cool  http://www.onlinemarktplatz.de/14177/twitter-fieber-bei-ebay/ ;_)\n'
 b'i have had 5 prank calls in 2 days, some the same, some COMPLETELY different... i really wanna know who it is! \n'
 b'is sad cause its time to get up \n'
 b'cant wait for the hillssssss ! \n'
 b'Lycos 14 year old mascot spider &quot;Rex&quot; passed away this morning.  RIP friend, you will be missed.  Her story: http://is.gd/rdsE\n'
 b"I don't know why, I just randomly 

In [12]:
#cache and prefetch
train_ds=raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [13]:
# validation dataset creation
val_ds = tf.keras.utils.text_dataset_from_directory(
    input_sentiment_train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
    )

Found 20002 files belonging to 2 classes.
Using 4000 files for validation.


In [14]:
#cache and prefetch
val_ds=val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model creation

In [15]:
def create_model():
  #text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
def create_model(learning_rate=2e-4):
  text_input_layer=tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
  preprocessing_layer=hub.KerasLayer(tfhub_handle_preprocess,name='preprocessing')
  bert_inputs=preprocessing_layer(text_input_layer)
  bert_model=hub.KerasLayer(tfhub_handle_encoder,name='bert_model',trainable=False)
  bert_output=bert_model(bert_inputs)

  bert_pooled_output = bert_output['pooled_output']
  drop = tf.keras.layers.Dropout(0.1)(bert_pooled_output)
  #dense1= tf.keras.layers.Dense(300,activation="relu")(drop)
  #dense2=tf.keras.layers.Dense(300,activation="relu")(dense1)
  #dense3=tf.keras.layers.Dense(300,activation="relu")(dense2)
  #dense4=tf.keras.layers.Dense(300,activation="relu")(dense3)
  #drop = tf.keras.layers.Dropout(0.4)(dense4)
  output=tf.keras.layers.Dense(1,activation=None,name="classifier")(drop )
  model = tf.keras.Model(inputs=text_input_layer,outputs=output)
  return model

In [16]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [23]:
tf.data.experimental.cardinality(train_ds).numpy()

501

In [17]:
epochs = 1
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
#steps_per_epoch=tweet_df.shape[0]
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [18]:
model=create_model()

In [19]:
model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

Results of fit using the imdb dataset

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = model.fit(x=train_ds,validation_data=val_ds,epochs=1)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
625/625 [==============================] - 5681s 9s/step - loss: 0.4762 - binary_accuracy: 0.7521 - val_loss: 0.3761 - val_binary_accuracy: 0.8384


result of using the tf.data.dataset in fit

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = model.fit(x=train_tweet_ds,validation_data=val_tweet_ds,epochs=1,batch_size=32,shuffle=True)

Use fit with padas list

In [ ]:
#print(f'Training model with {tfhub_handle_encoder}')
#history = model.fit(x=X_train,y=y_train,validation_split=0.2,epochs=1,batch_size=32,shuffle=True)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
  381/32000 [..............................] - ETA: 72:59:09 - loss: 0.9440 - binary_accuracy: 0.5062

KeyboardInterrupt: ignored

Use fit with tweet dataset

when using a train dataset of 200,00 samples , the model was taking long to complete training. The accuracy was close to 50% on the first epoch when 100
The total number of sample were changed to 10,000.

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = model.fit(x=train_ds,validation_data=val_ds,epochs=1)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
  1/501 [..............................] - ETA: 40:08:02 - loss: 0.7195 - binary_accuracy: 0.5625